# Case Study 1
---
<div style="margin: -10px 0 20px 0"><small>Author: Daniel Kończyk (18208152)</small></div>

This case study focuses on network analysis and modeling by using graphs taken from the `SNAP` repository.

The study uses the following software:
* Python v3.7
* matplotlib package v3.1
* numpy package v1.16
* networkx package v2.3

The notebook has been executed on the following environment:
* Linux and macOS (PC and laptop)
* 16GB RAM
* 4-core CPU

I've selected fairly large graphs for analysis to make this case study a bit more challenging, but unfortunately this also results in **fairly high computation time** in some parts of the study. These computations are generally CPU-intensive and memory usage should be fairly low (less than 8GB). They have been wrapped in timing code to clearly show the expected runtime and actual values have been put in comments to skip the compuation altogether, if needed. 

We start the notebook with importing all required packages and modules.


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import time
import datetime
import itertools
from collections import defaultdict
from functools import reduce

The graphs were downloaded from the `SNAP` repository and stored locally for convenience.  
* For directed graph **Stanford web graph** was chosen and downloaded from https://snap.stanford.edu/data/web-Stanford.html
* For undirected graph **Autonomous systems by Skitter** was chosen and downloaded from https://snap.stanford.edu/data/as-Skitter.html

The references to the downloaded and unpacked files are then stored in the following variables for later use:

In [2]:
directed_graph_file = "web-Stanford.txt"
undirected_graph_file = "email-Enron.txt"

##  “Broder et al” butterfly picture of the directed network

We start with computing Strongly Connected Components (SCC) in the directed graph of choice. While there are ready-made functions in networkx, I've decided to implement one of the known algorithms (*Kosaraju's algorithm*) and use my own simple data structure to represent the graph.

For the purpose of SCC computation, the graph is represented as an adjacency list, where each key represents a node and value represents its neighbours as a set of nodes.

In [3]:
G = defaultdict(set)
# Load data into the graph data structure
with open(directed_graph_file, "r") as ins:
    for line in ins:
        if not line.startswith("#"):
            u, v = [int(i) for i in line.split()]
            G[u].add(v)

As mentioned before, I chose to implement *Kosaraju's algorithm* to compute SCC in the directed graph. The complexity of this algorithm is $O(E+V)$. The main idea here is that the SCCs in graph G are the same as in the transpose of G (direction of every edge reversed).  
The implementation is much cleaner with recursive DFS, but this often fails in large graphs due to stack overflows, so I decided to implement an iterative DFS version from the start:

In [4]:
# Compute SCC
def scc(G):
    """
    Run Depth First Search to find out the finish times 
    to find each vertex in graph G.
    Pass the visited set in case we have a disconnected graph
    """
    visited, finish_times = set(), []
    for u in list(G.keys()):
        if u not in visited:
            finish_times.extend([v for v in dfs(G, u, visited)])
    
    # Reverse graph G
    GR = defaultdict(set)
    for u, nodes in G.items():
        for v in nodes:
            GR[v].add(u)
    
    """
    Compute strongly connected components by running Depth First Search
    on a reversed graph G, by exploring in reversed finish_times order.
    Pass the visited set to no explore the same components multiple times
    """
    visited, scc = set(), defaultdict(set)
    for u in reversed(finish_times):
        if u not in visited:
            scc[u].update([node for node in dfs(GR, u, visited)])
    return scc

# Run iterative DFS to prevent stack overflow in large graphs
def dfs(G, source, visited):
    stack, popped = [source], set()
    while stack:
        node = stack[-1]
        to_visit = G[node] - visited
        if node not in visited:
            visited.add(node)
            stack.extend(to_visit)
        else:
            stack.pop()
            if node not in popped:
                popped.add(node)
                yield node

Finally, we are ready to compute the SCC and extract the giant (largest SCC) component. Additionaly we extract the nodes that are not in the giant SCC to make further computations easier:

In [5]:
s = time.perf_counter()
components = sorted(scc(G).values(), key=len)
# largest SCC
giant = components[-1]
# nodes not in largest SCC
not_giant = reduce(set.union, components[:-1])
print("Computation time: {}".format(datetime.timedelta(seconds=int(time.perf_counter()-s))))

Computation time: 0:00:17


Having the components computed, we are ready to find the IN and OUT sections of the graph.  
* The size of an IN section can be found by iterating over the nodes outside the giant SCC and checking for intersection of their neighbours with the giant SCC
* The size of an OUT section can be found by iterating over the nodes in the giant SCC and checking for intersection of their neighbours with nodes outside the giant SCC

In [6]:
in_nodes = 0
for u in not_giant:
    if G[u].intersection(giant):
        in_nodes += 1

In [7]:
out_nodes = 0
for u in giant:
    if G[u].intersection(not_giant):
        out_nodes += 1

We can now print out the results for this section:

In [8]:
print("Nodes in the giant SCC: {}".format(len(giant)))
print("Nodes in the IN section of the graph: {}".format(in_nodes))
print("Nodes in the OUT section of the grap: {}".format(out_nodes))

Nodes in the giant SCC: 150532
Nodes in the IN section of the graph: 26239
Nodes in the OUT section of the grap: 2604


## The parameter β of the power-law degree distribution model

The next part of the study focuses on fitting a line to a log-log plot of the degree distribution of both graphs and computing its slope to determine the parameter $\beta$ of the power-law degree distribution model.

We start by loading both graphs from files, this time using `networkx` package:

In [9]:
s = time.perf_counter()
D = nx.read_edgelist(directed_graph_file, create_using=nx.DiGraph(), nodetype=int)
G = nx.read_edgelist(undirected_graph_file, create_using=nx.Graph(), nodetype=int)
print("Loading time: {}".format(datetime.timedelta(seconds=int(time.perf_counter()-s))))

NameError: name 'directed_graph' is not defined

The parameter $\beta$ can be easily determined from the CCDF, where $P(D_v>d)=\big(\frac{d}{d_{min}}\big)^{-\beta}$ and $\beta = \alpha-1$.  
A few functions to compute the necessary data are created:
* `gen_ccdf` computes CCDF from the degree view of a graph, returning a tuple with lists of $x$ and $y$ values, where $x$ are degrees of the graph and $y$ are the associated probabilities of the CCDF. This function adds 1 to each degree if the smallest degree is 0 to avoid problems with log function
* `plot_ccdf` plots the CCDF (x,y values) along with the fitted line (fit_coeffs)
* `fit_cdf` fits a line to a log-log of the CCDF by using *polyfit* function from numpy

In [ ]:
def gen_ccdf(degrees):
    hist = {}
    c = 0
    if min([d for _, d in degrees]) == 0:
        c = 1    
    for d in [d+c for _, d in degrees]:
        hist[d] = hist.get(d,0) + 1

    cumsum = 0
    ccdf = {}
    for d in sorted(hist.keys(), reverse=True):
        cumsum += hist[d]
        ccdf[d] = cumsum/len(degrees)
    x,y = zip(*(ccdf.items()))
    return np.array(x), np.array(y)
        
def plot_ccdf(x, y, fit_coeffs, title):
    plt.loglog(x,y)
    x1 = sorted(x)
    poly = np.poly1d(fit_coeffs)
    yfit = lambda x: np.exp(poly(np.log(x)))
    plt.loglog(x1,yfit(x1),"--",color="r")
    plt.ylim(1e-6)
    plt.xlabel("in-degree ($d$)")
    plt.ylabel("$P(D_v > d)$")
    plt.title(title)

def fit_ccdf(x, y):
    return np.polyfit(np.log(x), np.log(y), 1)

We are now ready to plot the CCDF of the directed graph and compute the slop and paramter $\beta$:

In [ ]:
x, y = gen_ccdf(D.in_degree())
coeffs = fit_ccdf(x,y)
plot_ccdf(x, y, coeffs, "In-degree distribution of the directed graph")
plt.show()
D_beta = -coeffs[0]
D_alpha = D_beta+1
print("Parameter β: {}".format(D_beta))
print("Parameter α: {}".format(D_alpha))

And the same plot of the CCDF and the fit line but for the undirected graph:

In [ ]:
x, y = gen_ccdf(G.degree())
coeffs = fit_ccdf(x,y)
plot_ccdf(x, y, coeffs, "Degree distribution of the undirected graph")
plt.show()
G_beta = -coeffs[0]
G_alpha = G_beta+1
print("Parameter β: {}".format(G_beta))
print("Parameter α: {}".format(G_alpha))

## Graph metrics

For most of the metrix `networkx` is used, but due to quite large graphs being selected, computing average path lengths was impossible on the full set of nodes (test run was executing for a few hours before I stopped it). Thus an auxillary function was provided to compute this metric only on a $N$ sample of nodes. The seed is used to get consistent results over multiple runs:

In [ ]:
def avg_shortest_path(G, N):
    visited = set()
    nodes = list(G.nodes())
    np.random.seed(12345)
    np.random.shuffle(nodes)
    avg = []
    for n in range(N):
        node = nodes[n]
        visited.add(node)
        lengths = []
        for s, l in nx.single_source_shortest_path_length(G,node).items():
            if s != node and G.is_directed() is True or s not in visited:
                lengths.append(l)
        if len(lengths) > 0:
            avg.append(np.mean(lengths))
    return np.mean(avg)

I was still curious about the actual values for the average path, so I decided to use a simple `R` script using `igraph` library:
```R
library(igraph)
d <- read.table("web-Stanford.txt",header=F,sep="\t",comment.char="#")
g = graph_from_edgelist(as.matrix(d[,1:2, drop = FALSE], directed = TRUE))
mean_distance(g, directed=TRUE)
```
Turns out R was able to compute it in about 1.5h and the value was XXX.

Nevertheless, I added pre-computed values for some of the variables in order to save time on the re-runs, since each computation takes about 15 minutes. We can just comment out actual code invocations if needed:

In [ ]:
s = time.perf_counter()
D_assortativity = nx.degree_pearson_correlation_coefficient(D)
# pre-computed value
# D_clustering = 0.5976304608024073
D_clustering = nx.average_clustering(D.to_undirected())
# pre-computed value
# D_shortest_path = 16.109514211378873
D_shortest_path = avg_shortest_path(D, 2000)
print("Computation time: {}".format(datetime.timedelta(seconds=int(time.perf_counter()-s))))

And now we can report the metrics for the **directed** graph:

In [ ]:
print("Degree Assortativity Coefficient:\t{}".format(D_assortativity))
print("Clustering Coefficient:\t\t\t{}".format(D_clustering))
print("Average path length (sampled):\t\t{}".format(D_shortest_path))

---
Similarly to the directed graph, I was crious about the actual values for the average path in the undirected graph too, which was not feasable to compute in Python, so I decided to use the same `R` script:
```R
library(igraph)
d <- read.table("as-skitter.txt",header=F,sep="\t",comment.char="#")
g <- graph_from_data_frame(d, directed = TRUE)
mean_distance(g, directed=TRUE)
```
R was able to compute it in about XXX and the value was XXX.

Again, I added pre-computed values for some of the variables in order to save time on the re-runs and can just comment the actual code invocations if needed:

In [ ]:
s = time.perf_counter()
G_assortativity = nx.degree_pearson_correlation_coefficient(G)
G_clustering = nx.average_clustering(G)
G_shortest_path = avg_shortest_path(G, 2000)
print("Computation time: {}".format(datetime.timedelta(seconds=int(time.perf_counter()-s))))

And now we can report the metrics for the **undirected** graph:

In [ ]:
print("Degree Assortativity Coefficient:\t{}".format(G_assortativity))
print("Clustering Coefficient:\t\t\t{}".format(G_clustering))
print("Average path length (sampled):\t\t{}".format(G_shortest_path))